In [30]:
from transformers.models.auto.modeling_auto import AutoModelForSequenceClassification

model_name = "google/bert_uncased_L-2_H-128_A-2"
batch_size = 512

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


loading configuration file https://huggingface.co/google/bert_uncased_L-2_H-128_A-2/resolve/main/config.json from cache at /home/ubuntu/.cache/huggingface/transformers/267e014e0691a01eec80d89bb2425a3d308d8cecfb1d6e87be0dc333713851b6.f7c0b2bc8a921bf36b061a6a20c9b520a23fb3f3cd8d356d1501424870e750a8
Model config BertConfig {
  "_name_or_path": "google/bert_uncased_L-2_H-128_A-2",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/google/bert_uncased_L-2_H-128_A-2/resolve/main/pytorch_model.bin fr

In [31]:
from pprint import pprint

pprint(model)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, element

In [32]:
from datasets.load import load_dataset, load_metric

dataset = load_dataset("glue", "qnli")
metric = load_metric("glue", "qnli")


Reusing dataset glue (/home/ubuntu/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 866.47it/s]


In [33]:
pprint(dataset)
pprint(metric)


DatasetDict({
    train: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 104743
    })
    validation: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 5463
    })
    test: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 5463
    })
})
Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_cor

In [34]:
from transformers.models.auto.tokenization_auto import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)


def preprocess_by(tokenizer, key, **kwargs):
    def preprocess(data):
        return tokenizer(data[key], **kwargs)

    return preprocess


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/google/bert_uncased_L-2_H-128_A-2/resolve/main/config.json from cache at /home/ubuntu/.cache/huggingface/transformers/267e014e0691a01eec80d89bb2425a3d308d8cecfb1d6e87be0dc333713851b6.f7c0b2bc8a921bf36b061a6a20c9b520a23fb3f3cd8d356d1501424870e750a8
Model config BertConfig {
  "_name_or_path": "google/bert_uncased_L-2_H-128_A-2",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https

In [35]:
encoded_dataset = dataset.map(
    preprocess_by(tokenizer, "sentence", truncation=True, max_length=512), batched=True
)


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-1c80317fa3b1799d.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-bdd640fb06671ad1.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3eb13b9046685257.arrow


In [36]:
from transformers.trainer_utils import IntervalStrategy
from transformers.training_args import TrainingArguments

args = TrainingArguments(
    "../" + model_name.replace("/", "-"),
    evaluation_strategy=IntervalStrategy.EPOCH,
    save_strategy=IntervalStrategy.EPOCH,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [37]:
import numpy as np

def compute_metrics_by(metric):
    def comp(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=1)
        return metric.compute(predictions=predictions, references=labels)
    return comp

In [38]:
from transformers.trainer import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],  # type: ignore
    eval_dataset=encoded_dataset["validation"],  # type: ignore
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_by(metric),
)

trainer.train()


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: question, sentence, idx. If question, sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/ubuntu/anaconda3/envs/word2vec/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 104743
  Num Epochs = 5
  Instantaneous batch size per device = 512
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 1
  Total optimization steps = 1025


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.659168,0.614864
2,No log,0.652540,0.618891
3,0.665900,0.650094,0.618708
4,0.665900,0.649487,0.617426
5,0.652700,0.648759,0.618708


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: question, sentence, idx. If question, sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5463
  Batch size = 512
Saving model checkpoint to ../google-bert_uncased_L-2_H-128_A-2/checkpoint-205
Configuration saved in ../google-bert_uncased_L-2_H-128_A-2/checkpoint-205/config.json
Model weights saved in ../google-bert_uncased_L-2_H-128_A-2/checkpoint-205/pytorch_model.bin
tokenizer config file saved in ../google-bert_uncased_L-2_H-128_A-2/checkpoint-205/tokenizer_config.json
Special tokens file saved in ../google-bert_uncased_L-2_H-128_A-2/checkpoint-205/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: question, sentenc

TrainOutput(global_step=1025, training_loss=0.6590895266649199, metrics={'train_runtime': 150.8364, 'train_samples_per_second': 3472.072, 'train_steps_per_second': 6.795, 'total_flos': 203986994646420.0, 'train_loss': 0.6590895266649199, 'epoch': 5.0})